# Voxel Based ICH Volume

In [35]:
import nibabel as nib
import numpy as np

def estimate_hemorrhage_volume(nifti_file):
    # Load NIfTI file
    img = nib.load(nifti_file)
    data = img.get_fdata()
    # print(data.shape)
    # print(img.header)

    # Threshold the image to extract hemorrhage region
    hemorrhage_mask = np.where(data > 0, 1, 0)
    sx, sy, sz = img.header.get_zooms()
    # print(sx*sy*sz) # volume

    # Calculate voxel size
    voxel_size = np.prod(img.header.get_zooms())
    
    # print(voxel_size)
    
    
    # print(hemorrhage_mask)
    
    print(f"Count of pixels in hemorrhage area: {np.sum(hemorrhage_mask)}")

    # Calculate hemorrhage volume in cubic millimeters
    hemorrhage_volume = np.sum(hemorrhage_mask) * voxel_size

    return hemorrhage_volume

# Example usage
nifti_file_path = '075.nii'
volume = estimate_hemorrhage_volume(nifti_file_path)
print(f"Hemorrhage volume: {volume} mm^3")
print(f"Hemorrhage volume: {volume/1000} cm^3")

Count of pixels in hemorrhage area: 97076
Hemorrhage volume: 74063.1103515625 mm^3
Hemorrhage volume: 74.0631103515625 cm^3


# Pre-processing

In [36]:
import os
import numpy as np
import nibabel as nib
from PIL import Image

def save_mask_images(nifti_file, output_folder):
    # Load NIfTI file
    img = nib.load(nifti_file)
    data = img.get_fdata()

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over each slice in the NIfTI data
    for slice_idx in range(data.shape[2]):
        # Extract the slice
        slice_data = data[:, :, slice_idx]

        # Threshold the slice to create the mask image
        mask = (slice_data > 0).astype(np.uint8) * 255

        # Resize the mask image to 512x512
        mask_image = Image.fromarray(mask)
        mask_image = mask_image.resize((512, 512))

        # Save the mask image as JPEG
        output_path = os.path.join(output_folder, f"mask_{slice_idx}.png")
        mask_image.save(output_path)

# Example usage
nifti_file_path = '075.nii'
output_folder_path = 'masks'
save_mask_images(nifti_file_path, output_folder_path)


In [37]:
import os
from PIL import Image

def calculate_hemorrhage_pixel_count(folder_path):
    total_count = 0

    # Iterate over each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            file_path = os.path.join(folder_path, filename)
            image = Image.open(file_path)

            # Convert the image to grayscale
            image = image.convert("L")

            # Convert the image to a numpy array
            mask_array = np.array(image)

            # Count the pixels corresponding to hemorrhage area
            hemorrhage_pixels = np.count_nonzero(mask_array)

            # Accumulate the count
            total_count += hemorrhage_pixels

    return total_count

# Example usage
folder_path = 'masks'
hemorrhage_pixel_count = calculate_hemorrhage_pixel_count(folder_path)
print(f"Count of pixels in hemorrhage area: {hemorrhage_pixel_count}")

voxel_size = 0.76293945 # Previous Calculation

hemorrhage_volume = hemorrhage_pixel_count * voxel_size
print(f"Hemorrhage volume: {hemorrhage_volume} mm^3")
print(f"Hemorrhage volume: {hemorrhage_volume/1000} cm^3")


Count of pixels in hemorrhage area: 97076
Hemorrhage volume: 74063.11004819999 mm^3
Hemorrhage volume: 74.06311004819999 cm^3
